# Parameter definition
Choose all the settings for the database creation. Input, output dir...

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_todas' #'colab_main_dataset'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['lfw', 'scface',  'enfsi', 'enfsi2015', 'chokepoint'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn','mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[22:12:59] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[22:12:59] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[22:12:59] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[22:12:59] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[22:12:59] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by pre

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2022-12-08 22:13:00.262919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 22:13:00.636103: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-08 22:13:01.890678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/cv2/../../lib64:
2022-12-08 22:13:01.890885: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

Creating Db file at /home/jmacarulla/video_resources/sql_database/test_video_todas.db


In [5]:
database.__dict__

{'db_name': 'test_video_todas',
 'input_dir': '/home/jmacarulla/video_resources/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/jmacarulla/video_resources/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7ff667f77d10>,
 'databases': [<sql_face.databases.LFW at 0x7ff667f16ed0>,
 'detector_names': ['mtcnn', 'mtcnn_serfiq'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
database.create_tables(serfiq=serfiq)

Creating image record from LFW: 13233it [00:01, 7431.18it/s]
Creating images in SCFace: 4160it [00:01, 4152.34it/s]
Creating Cropped Images for detector mtcnn:   0%|          | 0/94117 [00:00<?, ?it/s]2022-12-08 22:21:35.626271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jmacarulla/anaconda3/envs/sql-face/lib/python3.7/site-packages/cv2/../../lib64:
2022-12-08 22:21:35.626495: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-08 22:21:35.626543: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-NSS2B5R): /proc/driver/nvidia/version does not exist
2022-12-08 22:21:35.630808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Net

1/1 [==============================] - 0s 284ms/step


Creating Cropped Images for detector mtcnn:   0%|          | 1/94117 [00:05<153:44:47,  5.88s/it]

1/1 [==============================] - 0s 45ms/step


Creating Cropped Images for detector mtcnn:   0%|          | 2/94117 [00:08<103:39:26,  3.97s/it]

...

1/1 [==============================] - 0s 27ms/step


Creating Cropped Images for detector mtcnn:   0%|          | 15/94117 [00:30<43:14:59,  1.65s/it]

1/1 [==============================] - 0s 32ms/step


Creating Cropped Images for detector mtcnn:   0%|          | 16/94117 [00:31<43:34:06,  1.67s/it]

1/1 [==============================] - 0s 27ms/step


In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)